In [1]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="huggingface_hub")

In [5]:
!pip install datasets
!pip install scikit-learn
!pip install torch
!pip install numpy
!pip install pandas
!pip install transformers

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)


In [15]:
import pandas as pd
import numpy as np
import random
import datasets
from transformers import TrainingArguments, Trainer, BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
import torch

In [16]:
# Фиксирование рандома, чтобы результат был более менее воспроизводим
def seed_all(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False

seed_all(1234)

In [22]:
# Загрузка данных
BASE_DIR = "/home/alexandr/PycharmProjects/NLP_homework/homework3"
data = pd.read_csv(BASE_DIR + "/in_domain_train.csv", usecols=['sentence', 'acceptable'])
test_data = pd.read_csv(BASE_DIR + "/in_domain_dev.csv")
test_data = test_data[['sentence', 'acceptable']]

train_data, val_data = train_test_split(data, test_size=0.2)
train_data = train_data.reset_index()[['sentence', 'acceptable']]
val_data = val_data.reset_index()[['sentence', 'acceptable']]

train_text = train_data['sentence']
train_labels = train_data['acceptable']
val_text = val_data['sentence']
val_labels = val_data['acceptable']
test_text = test_data['sentence']
test_labels = test_data['acceptable']

# Новый раздел

In [23]:
train_data

,sentence,acceptable
0,Приближался нечеловеческие рев и топот.,0
1,В котором часу завтра утром начинается конфере...,1
2,Она сидела на диване рядом с мужем.,1
3,"Это были студенты, сами сведущие в эскимосском...",1
4,"При всем том я оптимист и думаю, что мой конфл...",1
...,...,...
6290,Она хочет поговорить с каким-нибудь хорошим сп...,1
6291,"При расставании я целовал три раза чудесные, с...",1
6292,"Парк в старом русле реки Турии, раскинувшиеся ...",0
6293,Страны НАТО хотят взять с Ирана обязательство ...,0


In [26]:
torch.__version__

'2.7.0+cu126'

In [25]:
# Загружаем нужную модель
model_checkpoint = "ai-forever/ruBert-base"
model = BertForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# if device.type == 'cuda':
#     model = model.to("cuda")
tokenizer = BertTokenizer.from_pretrained(model_checkpoint)

ImportError: 
BertForSequenceClassification requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.


In [11]:
# Данная модель принимает предложения длиной, не больше 512 токенов, поэтому сначала проверю,
# какая максимальная длина в train, val и test
seq_len_train = [len(str(i).split()) for i in train_data['sentence']]
seq_len_val = [len(str(i).split()) for i in val_data['sentence']]
seq_len_test = [len(str(i).split()) for i in test_data['sentence']]
max_seq_len = max(max(seq_len_test), max(seq_len_train), max(seq_len_val))
max_seq_len

26

In [12]:
tokens_train = tokenizer.batch_encode_plus(
    train_text.values,
    max_length = max_seq_len,
    padding = 'max_length',
    truncation = True
)
tokens_val = tokenizer.batch_encode_plus(
    val_text.values,
    max_length = max_seq_len,
    padding = 'max_length',
    truncation = True
)
tokens_test = tokenizer.batch_encode_plus(
    test_text.values,
    max_length = max_seq_len,
    padding = 'max_length',
    truncation = True
)

In [13]:
# Данный код оборачивает токенизированные текстовые данные в torch Dataset
class Data(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Data(tokens_train, train_labels)
val_dataset = Data(tokens_val, val_labels)
test_dataset = Data(tokens_test, test_labels)

In [14]:
# Функция для расчета метрики
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [24]:
# Параметры, которые будут использоваться для обучения
training_args = TrainingArguments(
    output_dir = BASE_DIR + '/fine-tuning_RuBERT_results', #Выходной каталог
    num_train_epochs = 3, #Кол-во эпох для обучения
    per_device_train_batch_size = 16, #Размер пакета для каждого устройства во время обучения
    per_device_eval_batch_size = 8, #Размер пакета для каждого устройства во время валидации
    weight_decay =5e-2, #Понижение весов
    load_best_model_at_end = True, #Загружать ли лучшую модель после обучения
    learning_rate = 2e-5, #Скорость обучения
    eval_strategy ='epoch', #Валидация после каждой эпохи (можно сделать после конкретного кол-ва шагов)
    logging_strategy = 'epoch', #Логирование после каждой эпохи
    save_strategy = 'epoch', #Сохранение после каждой эпохи
    gradient_accumulation_steps = 16,
    save_total_limit = 1,
    )

In [25]:
trainer = Trainer(model=model,
                  args = training_args,
                  train_dataset = train_dataset,
                  eval_dataset = val_dataset,
                  compute_metrics = compute_metrics)

In [26]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.513800,0.518274,0.771283,0.772944,0.977644,0.863326
2,0.387300,0.578172,0.790343,0.802909,0.949269,0.869976
3,0.262000,0.855307,0.787802,0.799278,0.951849,0.868917
4,0.186200,1.096061,0.792249,0.814286,0.931212,0.868833
5,0.129000,1.237774,0.785260,0.800000,0.945830,0.866824


TrainOutput(global_step=3935, training_loss=0.2956621104607588, metrics={'train_runtime': 20973.5907, 'train_samples_per_second': 1.501, 'train_steps_per_second': 0.188, 'total_flos': 420540883113000.0, 'train_loss': 0.2956621104607588, 'epoch': 5.0})

In [18]:
# Параметры, которые будут использоваться для обучения
training_args = TrainingArguments(
    output_dir = './results', #Выходной каталог
    num_train_epochs = 3, #Кол-во эпох для обучения
    per_device_train_batch_size = 8, #Размер пакета для каждого устройства во время обучения
    per_device_eval_batch_size = 8, #Размер пакета для каждого устройства во время валидации
    weight_decay =0.01, #Понижение весов
    logging_dir = './logs', #Каталог для хранения журналов
    load_best_model_at_end = True, #Загружать ли лучшую модель после обучения
    learning_rate = 5e-5, #Скорость обучения
    eval_strategy ='epoch', #Валидация после каждой эпохи (можно сделать после конкретного кол-ва шагов)
    logging_strategy = 'epoch', #Логирование после каждой эпохи
    save_strategy = 'epoch', #Сохранение после каждой эпохи
    save_total_limit = 1,
    lr_scheduler_type='cosine'
    )

In [19]:
trainer = Trainer(model=model,
                  args = training_args,
                  train_dataset = train_dataset,
                  eval_dataset = val_dataset,
                  compute_metrics = compute_metrics)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.532500,0.535817,0.770648,0.780044,0.947788,0.855773


In [1]:
# Оцениваем точность на проверенном наборе val_data
eval_results = trainer.evaluate()
print(f'Точность в наборе для проверки: {eval_results["eval_accuracy"]:.3f}')

NameError: name 'trainer' is not defined

In [ ]:
# Сохранение обученной модели
model_path = "fine-tune-bert"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

In [ ]:
# Написание функции для получения предикта
def get_prediction():
    test_pred = trainer.predict(test_dataset)
    labels = np.argmax(test_pred.predictions, axis = -1)
    return labels
pred = get_prediction()

In [ ]:
# Проверка полученного результата
print(classification_report(test_labels, pred))
print(f1_score(test_labels, pred))